# Agent Observability

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/datascienceworld-kan/vinagent-docs/blob/main/docs/tutorials/get_started/observability.ipynb)

Vinagent provides a local MLflow dashboard that can be used to visualize the intermediate messsages of each query. Therefore, it is an important feature for debugging.

- Engineer can trace the number of tokens, execution time, type of tool, and status of exection.
- Based on tracked results, Agent developers can indentify inefficient steps, optimize agent components like tools, prompts, agent description, agent skills, and LLM model.
- Accelerate process of debugging and improving the agent's performance.


In [ ]:
%pip install vinagent

## Start MLflow UI

MLflow offers an local UI, which connets to mlflow server understreaming. This UI comprises all experients from conversations between user and agent. To start this UI, let's run this command on `terminal/command line interface` in your computer:

In [ ]:
mlflow ui

An MLflow dashboard starts, which can be accessed at [http://localhost:5000](http://localhost:5000).


## Initialize Experiment

Initialize an experiment to auto-log messages for agent

In [1]:
import mlflow
from vinagent.mlflow import autolog

# Enable Vinagent autologging
autolog.autolog()

# Optional: Set tracking URI and experiment
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("agent-dev")

2025/07/16 19:07:59 INFO mlflow.tracking.fluent: Experiment with name 'agent-dev' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/900548938729976750', creation_time=1752667679250, experiment_id='900548938729976750', last_update_time=1752667679250, lifecycle_stage='active', name='agent-dev', tags={}>

After this step, an experiment named `agent-dev` is initialized. An observability and tracing feature are automatically registered for each query to the agent without requiring any changes to the original invocation code.

## Observability and Tracing

A default MLflow dashboard is launched to display the experiment results, within the Jupyter Notebook, making it convenient for agent developers to test and optimize their agent design directly. Every query is now tracked under the experiment named `agent-dev`.

In [ ]:
%%writefile .env
TOGETHER_API_KEY="Your together API key"
TAVILY_API_KEY="Your Tavily API key"

In [2]:
from langchain_together import ChatTogether 
from vinagent.agent.agent import Agent
from dotenv import load_dotenv
load_dotenv()

llm = ChatTogether(
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo-Free"
)

agent = Agent(
    description="You are an Expert who can answer any general questions.",
    llm = llm,
    skills = [
        "Searching information from external search engine\n",
        "Summarize the main information\n"],
    tools = ['vinagent.tools.websearch_tools'],
    tools_path = 'templates/tools.json',
    memory_path = 'templates/memory.json'
)

result = agent.invoke(query="What is the weather today in Ha Noi?")

INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:vinagent.register.tool:Registered search_api:
{'tool_name': 'search_api', 'arguments': {'query': {'type': 'Union[str, dict[str, str]]', 'value': '{}'}}, 'return': 'Any', 'docstring': 'Search for an answer from a query string\n    Args:\n        query (dict[str, str]):  The input query to search\n    Returns:\n        The answer from search query', 'dependencies': ['os', 'dotenv', 'tavily', 'dataclasses', 'typing'], 'module_path': 'vinagent.tools.websearch_tools', 'tool_type': 'module', 'tool_call_id': 'tool_5d2b53b1-b012-48ac-83cd-bc9d3c115c50'}
INFO:vinagent.register.tool:Completed registration for module vinagent.tools.websearch_tools
INFO:vinagent.agent.agent:I'am chatting with unknown_user
INFO:httpx:HTTP Request: POST https://api.together.xyz/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:{'tool_name': 'search_api', 'tool_type': 'module', 'arguments': {'query': 'Ha Noi weather today'

[Trace(trace_id=74105e3849a041e09eb828d73dfbf515), Trace(trace_id=6ec5f26c820340ad9a2b5d3091771eb6)]

*Note:*
    You are able to access the dashboard at http://localhost:5000/ and view logs of aformentioned query by accessing to `agent-dev` and click to `Traces` tab on the last of header navigation bar of `agent-dev` experiment.